In [1]:
from reachy_sdk import ReachySDK

Connect to Reachy. If you're working directly on the robot, use

```python
host='localhost'
```

If not, just get Reachy's IP address with

```bash
$ ifconfig
```

and replace the host argument zith the IP address.

In [2]:
reachy = ReachySDK(host='localhost')

Check if you see the five joints of the head.

In [3]:
reachy.head

<Head joints=<Holder
	<Joint name="l_antenna" pos="-0.15" mode="compliant">
	<Joint name="r_antenna" pos="1.03" mode="compliant">
	<Joint name="neck_disk_top" pos="-50.26" mode="compliant">
	<Joint name="neck_disk_middle" pos="-56.95" mode="compliant">
	<Joint name="neck_disk_bottom" pos="-53.34" mode="compliant">
>>

Use look_at to see if the zero has been actually done.

## Checking Orbita

In [4]:
reachy.turn_on('head')

Make Reachy look forward.

In [5]:
reachy.head.look_at(x=0.5, y=0, z=0, duration=1.0)

If this didn't work, it's likely that the zeros of Orbita have not been set. To do that use [orbita_zero.py](https://github.com/pollen-robotics/reachy_controllers/blob/master/setup/orbita_zero.py).

Next, reproduce the look_at sequence of the [documentation](https://pollen-robotics.github.io/reachy-2021-docs/sdk/first-moves/head/#orbita-look_at-method).

In [6]:
import time

look_right = reachy.head.look_at(x=0.5, y=-0.5, z=0.1, duration=1.0)
time.sleep(0.5)
look_down = reachy.head.look_at(x=0.5, y=0, z=-0.4, duration=1.0)
time.sleep(0.5)
look_left = reachy.head.look_at(x=0.5, y=0.3, z=-0.3, duration=1.0)
time.sleep(0.5)
look_front = reachy.head.look_at(x=0.5, y=0, z=0, duration=1.0)

Make the head follow the end-effector. It is a good opportunity to check the forward kinematics of each arm as well.

**Press the square button 'interrupt the kernel' when you want to stop this.**

With the right arm:

In [8]:
try:
    x, y, z = reachy.r_arm.forward_kinematics()[:3,-1] # We want the translation part of Reachy's pose matrix
    reachy.head.look_at(x=x, y=y, z=z-0.05, duration=1.0) # There is a 5cm offset on the z axis

    time.sleep(0.5)

    while True:
        x, y, z = reachy.r_arm.forward_kinematics()[:3,-1]
        gp_dic = reachy.head._look_at(x, y, z - 0.05)
        reachy.head.neck_disk_bottom.goal_position = gp_dic[reachy.head.neck_disk_bottom]
        reachy.head.neck_disk_middle.goal_position = gp_dic[reachy.head.neck_disk_middle]
        reachy.head.neck_disk_top.goal_position = gp_dic[reachy.head.neck_disk_top]
        time.sleep(0.01)
except AttributeError:
    print('Reachy has no right arm!')

With the left arm:

In [19]:
try:
    x, y, z = reachy.l_arm.forward_kinematics()[:3,-1] # We want the translation part of Reachy's pose matrix
    reachy.head.look_at(x=x, y=y, z=z-0.05, duration=1.0) # There is a 5cm offset on the z axis

    time.sleep(0.5)

    while True:
        x, y, z = reachy.l_arm.forward_kinematics()[:3,-1]
        gp_dic = reachy.head._look_at(x, y, z - 0.05)
        reachy.head.neck_disk_bottom.goal_position = gp_dic[reachy.head.neck_disk_bottom]
        reachy.head.neck_disk_middle.goal_position = gp_dic[reachy.head.neck_disk_middle]
        reachy.head.neck_disk_top.goal_position = gp_dic[reachy.head.neck_disk_top]
        time.sleep(0.01)
except AttributeError:
    print('Reachy has no left arm!')

## Checking the antennas

Check that the l_antenna is actually the left and that r_antenna is the right.

In [9]:
reachy.head.l_antenna.goal_position = -40

In [10]:
reachy.head.l_antenna.goal_position = 0

In [11]:
reachy.head.r_antenna.goal_position = -40

In [12]:
reachy.head.r_antenna.goal_position = 0

Reproduce the sad and happy movements from the [documentation](https://pollen-robotics.github.io/reachy-2021-docs/sdk/first-moves/head/#antennas).

In [13]:
def happy_antennas():
    reachy.head.l_antenna.speed_limit = 0.0
    reachy.head.r_antenna.speed_limit = 0.0
    
    for _ in range(9):
        reachy.head.l_antenna.goal_position = 10.0
        reachy.head.r_antenna.goal_position = -10.0

        time.sleep(0.1)

        reachy.head.l_antenna.goal_position = -10.0
        reachy.head.r_antenna.goal_position = 10.0

        time.sleep(0.1)
    
    reachy.head.l_antenna.goal_position = 0.0
    reachy.head.r_antenna.goal_position = 0.0
        
def sad_antennas():
    reachy.head.l_antenna.speed_limit = 70.0
    reachy.head.r_antenna.speed_limit = 70.0
    
    reachy.head.l_antenna.goal_position = 140.0
    reachy.head.r_antenna.goal_position = -140.0
    
    time.sleep(5.0)
    
    reachy.head.l_antenna.goal_position = 0.0
    reachy.head.r_antenna.goal_position = 0.0

In [14]:
happy_antennas()

In [15]:
sad_antennas()

Once you're done using the antennas, make Reachy look forward.

In [ ]:
reachy.head.look_at(x=0.5, y=0, z=0, duration=1.0)

and turn off the motors.

In [17]:
reachy.turn_off('head')

## Checking the motorized zooms

Check out each of the three zoom levels defined on both cameras.

At the same time, use [view_cam.py](https://github.com/pollen-robotics/reachy_controllers/blob/master/examples/view_cam.py) in *~/reachy_ws/src/reachy_controllers/examples* to visualize the cameras and check if the images are clear enough.

To view the left camera, just type:
```bash
python3 ~/reachy_ws/src/reachy_controllers/examples/view_cam.py left ros
```

First, perform a homing.

In [15]:
from reachy_sdk.camera import ZoomLevel

# reachy.left_camera.zoom_homing()
reachy.right_camera.zoom_homing()
reachy.left_camera.zoom_homing()

True

Then, test each zoom level.

In [16]:
reachy.right_camera.zoom_level = ZoomLevel.INTER
reachy.left_camera.zoom_level = ZoomLevel.INTER

In [14]:
reachy.left_camera.zoom_level = ZoomLevel.OUT

In [15]:
reachy.fans.l_antenna_fan.off()

In [16]:
reachy.left_camera.zoom_speed

10000

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/reachy/dev/reachy-sdk/reachy_sdk/camera.py", line 132, in poll_img
    for resp in self._stub.StreamImage(stream_request):
  File "/home/reachy/.local/lib/python3.8/site-packages/grpc/_channel.py", line 426, in __next__
    return self._next()
  File "/home/reachy/.local/lib/python3.8/site-packages/grpc/_channel.py", line 826, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "{"created":"@1620394638.490089613","description":"Error received from peer ipv6:[::1]:50057","file":"src/core/lib/surface/call.cc","file_line":1067,"grpc_message":"Socket closed","grpc_status":14}"
